<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/state_farm_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# State Farm Pre-employment Assessment
### *Model-based supervised learning binary classification task*
Your work will be evaluated in the following areas:
- The appropriateness of the steps you took
- The complexity of your models
- The performance of each model on the test set (using AUC)
- The organization and readability of your code
- The write-up comparing the models
---

## Step 1 - Clean and prepare your data
The data in this exercise have been simulated to mimic real, dirty data. Please clean the data with whatever method(s) you believe to be best/most suitable. Success in this exercise typically involves feature engineering and avoiding data leakage. You may create new features. However, you may not add or supplement with external data. 

We will begin by importing relevant libraries.

In [ ]:
import pandas as pd

Next, we use the pandas library to load our CSV data.

In [2]:
train_data = pd.read_csv("exercise_40_train.csv")
test_data = pd.read_csv("exercise_40_test.csv")

In [3]:
train_data.head()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,0,0.165254,18.060003,Wed,1.077380,-1.339233,-1.584341,0.0062%,0.220784,1.816481,...,-0.397427,0.909479,no,5.492487,NaN,10.255579,7.627730,0,yes,104.251338
1,1,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064%,1.192441,3.513950,...,0.656651,9.093466,no,3.346429,4.321172,NaN,10.505284,1,yes,101.230645
2,1,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04%,0.952323,0.782974,...,2.059615,0.305170,no,4.456565,NaN,8.754572,7.810979,0,yes,109.345215
3,0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057%,-0.520756,1.825586,...,0.899392,5.971782,no,4.100022,1.151085,NaN,9.178325,1,yes,103.021970
4,0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109%,-0.732739,2.151990,...,3.003595,1.046096,yes,3.234033,2.074927,9.987006,11.702664,0,yes,92.925935


In [9]:
train_data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Columns: 101 entries, y to x100
dtypes: float64(86), int64(3), object(12)
memory usage: 30.8+ MB


<ipython-input-9-7d059bb87f2f>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  train_data.info(null_counts=True)


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      10000 non-null  float64
 1   x2      10000 non-null  float64
 2   x3      10000 non-null  object 
 3   x4      10000 non-null  float64
 4   x5      9398 non-null   float64
 5   x6      10000 non-null  float64
 6   x7      10000 non-null  object 
 7   x8      10000 non-null  float64
 8   x9      10000 non-null  float64
 9   x10     10000 non-null  float64
 10  x11     8671 non-null   float64
 11  x12     10000 non-null  float64
 12  x13     10000 non-null  float64
 13  x14     7572 non-null   float64
 14  x15     10000 non-null  float64
 15  x16     7247 non-null   float64
 16  x17     10000 non-null  float64
 17  x18     10000 non-null  float64
 18  x19     10000 non-null  object 
 19  x20     10000 non-null  float64
 20  x21     10000 non-null  float64
 21  x22     9387 non-null   float64
 22

There are 40,000 examples in the training set and 10,000 in the test set. The first column in the training set is the label with a header of `y`. The remaining 100 columns are features with a header of `x` followed by the feature number (i.e.: feature 10 column is labeled `x10`). There are 12 object dtype columns, 2 int64, and 86 float64.

In [6]:
train_data.select_dtypes(include=['int64'])

,y,x59,x98
0,0,0,0
1,1,0,1
2,1,0,0
3,0,0,1
4,0,0,0
...,...,...,...
39995,0,0,1
39996,0,0,0
39997,0,0,1
39998,0,0,0


In [7]:
train_data.select_dtypes(include=['object'])

,x3,x7,x19,x24,x31,x33,x39,x60,x65,x77,x93,x99
0,Wed,0.0062%,$-908.650758424405,female,no,Colorado,5-10 miles,August,farmers,mercedes,no,yes
1,Friday,0.0064%,$-1864.9622875143,male,no,Tennessee,5-10 miles,April,allstate,mercedes,no,yes
2,Thursday,-8e-04%,$-543.187402955527,male,no,Texas,5-10 miles,September,geico,subaru,no,yes
3,Tuesday,-0.0057%,$-182.626380634258,male,no,Minnesota,5-10 miles,September,geico,nissan,no,yes
4,Sunday,0.0109%,$967.007090837503,male,yes,New York,5-10 miles,January,geico,toyota,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,Sun,-0.0085%,$3750.51991954505,female,no,NaN,5-10 miles,July,farmers,NaN,no,yes
39996,Thursday,0.0077%,$448.867118077561,male,yes,Illinois,5-10 miles,July,progressive,ford,no,yes
39997,Monday,-0.0216%,$834.95775080472,male,yes,NaN,5-10 miles,August,geico,ford,no,yes
39998,Tuesday,1e-04%,$-48.1031003332715,male,no,Ohio,5-10 miles,December,farmers,NaN,no,yes


The int64 `[x59, x98]` comprise two feature columns and one label column of [0,1] and the 12 object dtypes `[x3, x7, x19, x24, x31, x33, x39, x60, x65, x77, x93, x99]` contain dates, percentages, currency, sex, yes/no, states, distance in miles, month, insurance carrier, vehicle make, and two more yes/no columns. The rest of the columns contain float64 numbers or NaN.

In [20]:
train_data.columns[train_data.isnull().sum() != 0]

Index(['x5', 'x11', 'x14', 'x16', 'x22', 'x24', 'x26', 'x30', 'x33', 'x38',
       'x41', 'x42', 'x44', 'x45', 'x49', 'x52', 'x54', 'x55', 'x57', 'x61',
       'x63', 'x64', 'x67', 'x68', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79',
       'x80', 'x83', 'x85', 'x86', 'x88', 'x89', 'x91', 'x92', 'x94', 'x95',
       'x96', 'x99'],
      dtype='object')

These are all the columns that have null values which will need to be addressed during feature engineering. The following are object dtype columns with null values: `[x24, x33, x77, x99]` corrresponding to [male/female, state, vehicle make, yes/no] while the rest are float64 dtype with null values.

Let's count the category column contents `[x3, x24, x31, x33, x39, x60, x65, x77, x93, x99]`

In [21]:
columns = ['x3', 'x24', 'x31', 'x33', 'x39', 'x60', 'x65', 'x77', 'x93', 'x99']
for col in columns:
    print(train_data[col].value_counts())

Wednesday    4930
Monday       4144
Friday       3975
Tuesday      3915
Sunday       3610
Saturday     3596
Tue          2948
Thursday     2791
Mon          2200
Wed          2043
Sat          1787
Thur         1643
Fri          1620
Sun           798
Name: x3, dtype: int64
female    18158
male      17986
Name: x24, dtype: int64
no     34022
yes     5978
Name: x31, dtype: int64
California        3393
Texas             2252
Florida           1802
New York          1714
Illinois          1240
Pennsylvania      1233
Ohio              1114
Michigan           982
Georgia            918
North Carolina     910
New Jersey         870
Virginia           791
Washington         750
Tennessee          690
Indiana            674
Arizona            665
Massachusetts      638
Wisconsin          635
Missouri           634
Minnesota          611
Maryland           581
Alabama            554
Colorado           530
Louisiana          501
South Carolina     491
Kentucky           478
Oregon             45

- We will need to combine duplicate days in `x3` such as Wednesday and Wed during our feature engineering.
- `x31` is imbalanced with only 15% yes and 85% no.
- `x39` has only one value and can safely be dropped.
- `x93` is also imbalanced with only 11% yes and 89% no.
- `x99` has only one value, other than NaN and can safely be dropped.

### Feature Engineering To-Do List
- We will need to combine duplicate days in `x3` such as Wednesday and Wed during our feature engineering.
- `x31` is imbalanced with only 15% yes and 85% no.
- `x39` has only one value and can safely be dropped.
- `x93` is also imbalanced with only 11% yes and 89% no.
- `x99` has only one value, other than NaN and can safely be dropped.
- Address null values in object columns [x24, x33, x77] corrresponding to [male/female, state, vehicle make] while the rest are float64 dtype.